In [1]:
import torch
from torch import nn
import tools
import pandas
import data_process
import MyTransformer

In [2]:
data  = pandas.read_csv('./motionClassify.csv')
vocab = data_process.gen_vocab(data)
data_train  =  data_process.gen_dataset(data[:40000],vocab)
data_test = data_process.gen_dataset(data[40000:],vocab)
Batch_size = 64
train_iter = torch.utils.data.DataLoader(data_train,Batch_size,shuffle=True)
test_iter = torch.utils.data.DataLoader(data_test,Batch_size,shuffle=True)

In [3]:
num_hiddens, num_layers, dropout =  768, 12, 0.1#num_hidden 其实是embedding_size或者说embedding_dim
batch_size, num_steps = 64, 10
lr, num_epochs, device = 0.005, 300, tools.try_gpu()
key_size=query_size=value_size=ffn_num_input=norm_shape=num_hiddens
ffn_num_hiddens, num_heads =  64,12

In [4]:
class transformerEncoder(nn.Module):
    def __init__(self,vocab_size,key_size, query_size, value_size, num_hiddens,
    norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,
    num_layers, dropout):
        super().__init__()
        self.encoder = MyTransformer.TransformerEncoder(
            vocab_size, key_size, query_size, value_size, num_hiddens,
            norm_shape, ffn_num_input, ffn_num_hiddens, num_heads,
            num_layers, dropout)
        self.dense = nn.Linear(num_hiddens,2)
    def forward(self,X):
        enc_X = self.encoder(X)
        return self.dense(enc_X[:,-1,:])

In [5]:
net = transformerEncoder(len(vocab),key_size,query_size,value_size,num_hiddens,
                         norm_shape,ffn_num_input,ffn_num_hiddens,num_heads,num_layers
                         ,dropout)

In [6]:
net.load_state_dict(torch.load('models/model1.pth'))

/tmp/ipykernel_966721/850103411.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  net.load_state_dict(torch.load('models/model1.pth'))


<All keys matched successfully>

In [6]:
optimizer = torch.optim.SGD(net.parameters(),lr)

tools.train(net,train_iter,device,optimizer,torch.nn.CrossEntropyLoss())

 16%|█▌        | 100/625 [01:13<08:21,  1.05it/s]

epoch1,batch100,loss = 0.7448557615280151


 32%|███▏      | 200/625 [02:27<06:45,  1.05it/s]

epoch1,batch200,loss = 0.6930150985717773


 48%|████▊     | 300/625 [03:41<05:10,  1.05it/s]

...

 80%|████████  | 500/625 [06:09<01:59,  1.05it/s]

epoch1,batch500,loss = 0.5411311984062195


 96%|█████████▌| 600/625 [07:23<00:23,  1.05it/s]

epoch1,batch600,loss = 0.5817689299583435


100%|██████████| 625/625 [07:40<00:00,  1.36it/s]


In [7]:
accuracy=tools.test(net,test_iter,device)

100%|██████████| 157/157 [00:36<00:00,  4.34it/s]

accuracy = 0.7240999937057495


In [10]:
torch.save(net.state_dict(),'./models/model1.pth')